# Process the Customers Data
- 1.Ingest the data into data lakehouse -- bronze_customers
- 2.Perform data quality checks and transform the data as required -- silver_customers_clean
- 3.Apply changes to the customers data -- silver_customers

In [0]:
%sql
create or refresh streaming table bronze_customers 
comment 'raw customers data ingested from the source system operational data'
tblproperties('quality'= 'bronze')
as 
select * ,
  current_timestamp() as ingestion_ts,
  _metadata.file_path as file_path
from cloud_files(
  '/Volumes/circuitbox/landing/operational_data/customers/',
  'json',
  map('cloudFiles.inferColumnTypes','true')
)

In [0]:
create or refresh streaming table silver_customers_clean(
  constraint valid_customer_id expect(customer_id is not null) on violation fail update,
  constraint valid_customer_name expect(customer_name is not null) on violation drop row,
  constraint valid_telephone expect(length(telephone) >= 10) ,
  constraint valid_email expect(email is not null),
  constraint valid_date_of_birth expect(date_of_birth>= '1920-01-01')
)
comment 'Cleaned customer data'
tblproperties('quality'= 'silver')
as 
SELECT customer_id,
customer_name,
cast(date_of_birth as date) as date_of_birth,
telephone,
email,
cast(created_date as date) as created_date
FROM stream(live.bronze_customers)

In [0]:
create or refresh streaming table silver_customer
comment 'this table contains SCD type1 details'
TBLPROPERTIES ('quality' = 'silver');


In [0]:

CREATE FLOW CUSTOMER_CDC AS AUTO CDC INTO silver_customer
FROM STREAM(LIVE.silver_customers_clean)
KEYS (customer_id)
SEQUENCE BY created_date
STORED AS SCD TYPE 1;
